### Are factor loadings stable?


In our PCA framework, which is a cross-sectional regression run at each time $t$, our asset returns evolve
following:

$$
Y_{i,t} = \beta_{i,t,0} + \sum_{j+1}^{M}\beta_{i,t,j}X_{t,j} + e_{i,t}
$$

There are $N$ assets returns spanning across $T$ time steps, $t, t+1, .. T$. The asset return matrix $Y$ can be explained via  linear combination of $J$ factors (in this case, principal componnets). $X_{t,j}$ represents the $j^{th}$ PC as estimated at time $t$ using the entire time-series available, while $\beta_{i,t,j}$ is the factor loading for asset $i$, factor $j$ at time $t$.

In the estimation of our model, the time-dynamics of the factor loadings are not implicit. We estimate them by running cross-sectional regression seperately at each time point.

We will perform two exercises.

Firstly, we will look at the difference for the parameters for a few fundamentally different assets.
1. CHRW.OQ
2. MCD.N
2. BAX.N
3. AMAT.OQ

In [11]:
assets = ['CHRW.OQ', 'MCD.N', 'BAX.N', 'AMAT.OQ', 'UPS.N']

In [12]:
import os
import numpy as np
import datetime as dt
from examples.example_executor_mvp_v3 import Executor
import matplotlib.pyplot as plt
import pandas as pd

In [13]:
START_DATE = '2005-12-31'
# this is included for data, but there is no leakage
END_DATE = START_DATE_TEST = '2023-06-04'
END_DATE_TEST = '2023-07-20'

PATH_DATA = fr'C:\Users\serge\IdeaProjects\portfolio_management\models\data\csv'
PATH_API_KEYS = r'C:\Users\serge\OneDrive\reuters\apikeys.csv'
PATH_SAVE_PLOTS = os.path.join(r'results\figures')
PATH_CONFIG = r'C:\Users\serge\IdeaProjects\portfolio_management\notebooks\examples\config_example.yaml'

In [ ]:
betas = {}

while END_DATE < '2024-04-21':

    print(END_DATE)

    executor = Executor(START_DATE, END_DATE, PATH_DATA, PATH_API_KEYS, PATH_CONFIG)

    executor._load_data()
    executor._preprocess()
    executor._fit_factor_model()

    idx = np.isin(executor.linear_model.assets_estimated, assets)
    idx = np.where(idx == True)[0]

    _betas = executor.linear_model.factor_loadings[:,idx]

    betas[END_DATE] = _betas

    h = executor.portfolio.horizon
    END_DATE = str((dt.datetime.strptime(END_DATE, '%Y-%m-%d') + dt.timedelta(days=h)).date())


2023-06-04


2024-10-24 23:15:26,085 P[21204] [MainThread 11488] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2024-10-24 23:15:26,085 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:15:26,085 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:15:26,085 P[21204] [MainThread 11488] Port number was not identified, cannot send any request


[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\WM.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\REG.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\BIO.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\IVZ.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\EXPE.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\GEN.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\DAY.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_managem

2024-10-24 23:15:40,557 P[21204] [MainThread 11488] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2024-10-24 23:15:40,557 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:15:40,557 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:15:40,557 P[21204] [MainThread 11488] Port number was not identified, cannot send any request


[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\WM.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\REG.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\BIO.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\IVZ.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\EXPE.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\GEN.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\DAY.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_managem

...

[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\WM.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\REG.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\BIO.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\IVZ.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\EXPE.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\GEN.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\DAY.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_managem

2024-10-24 23:17:19,939 P[21204] [MainThread 11488] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2024-10-24 23:17:19,939 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:17:19,939 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:17:19,939 P[21204] [MainThread 11488] Port number was not identified, cannot send any request


[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\WM.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\REG.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\BIO.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\IVZ.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\EXPE.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\GEN.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\DAY.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_managem

2024-10-24 23:17:37,593 P[21204] [MainThread 11488] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2024-10-24 23:17:37,593 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:17:37,593 P[21204] [MainThread 11488] Error on handshake url http://127.0.0.1:None/api/handshake : InvalidURL("Invalid port: 'None'")
2024-10-24 23:17:37,593 P[21204] [MainThread 11488] Port number was not identified, cannot send any request


[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\WM.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\REG.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\BIO.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\IVZ.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\EXPE.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\GEN.OQ.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_management\\models\\data\\csv\\prices\\DAY.N.csv'
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_managem

In [ ]:
res = {}

for a in range(len(assets)):
    times = list(betas.keys())
    _table = pd.DataFrame(\
        data=np.array([betas[i][:,a] for i in times]),\
        columns=['intercept', 'pc1', 'pc2', 'pc3']\
    )
    _table['date'] = times
    res[assets[a]] = _table

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(11,8))
plt.suptitle(assets[0])
axs[0,0].plot(res[assets[0]].iloc[:,-1],res[assets[0]].iloc[:,0],
              marker='o', color='red')
axs[0,1].plot(res[assets[0]].iloc[:,-1],res[assets[0]].iloc[:,1],
              marker='o', color='red')
axs[1,0].plot(res[assets[0]].iloc[:,-1],res[assets[0]].iloc[:,2],
              marker='o', color='red')
axs[1,1].plot(res[assets[0]].iloc[:,-1],res[assets[0]].iloc[:,3],
              marker='o', color='red')


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(11,8))
plt.suptitle(assets[0])
axs[0,0].plot(res[assets[0]].iloc[:,-1],np.abs(res[assets[0]].iloc[:,0]),
              marker='o', color='red')
axs[0,1].plot(res[assets[0]].iloc[:,-1],np.abs(res[assets[0]].iloc[:,1]),
              marker='o', color='red')
axs[1,0].plot(res[assets[0]].iloc[:,-1],np.abs(res[assets[0]].iloc[:,2]),
              marker='o', color='red')
axs[1,1].plot(res[assets[0]].iloc[:,-1],np.abs(res[assets[0]].iloc[:,3]),
              marker='o', color='red')

res[assets[0]]